# Application of Machine Learning Algorithms to GRS Data in Python

**By Tom Sander**

---

## Overview

This notebook demonstrates how to apply **unsupervised machine learning** (specifically K-Means clustering) to **Lunar Prospector Gamma-Ray Spectrometer (GRS)** data. By the end of this tutorial, you will:

1. Load and preprocess NASA planetary data
2. Understand why data normalization is critical for ML
3. Use the **Elbow Method** to determine the optimal number of clusters
4. Create an automated **geochemical map** of the Moon
5. Interpret the results in terms of known lunar geology

---

## 1. Import Packages

We'll use the following libraries:
- **Polars**: A fast DataFrame library for data manipulation (similar to Pandas but faster) [polars-website](https://pola.rs/)
- **Matplotlib**: For creating visualizations [matplotlib-website](https://matplotlib.org/)
- **Scikit-learn**: For machine learning algorithms (imported later) [scikit-learn-website](https://scikit-learn.org/stable/index.html)

In [ ]:
# Data manipulation library - Polars is a high-performance alternative to Pandas
import polars as pl

# Visualization library for creating plots and maps
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection

---

## 2. Load Data

The Lunar Prospector GRS data is stored in NASA's PDS (Planetary Data System) format. The `.tab` file contains the actual data in a fixed-width, whitespace-separated format with 61 columns per record.

**Data Source:** [NASA PDS Geosciences Node](https://pds-geosciences.wustl.edu/)

### Custom Data Loader Function

NASA PDS files have a specific format that requires custom parsing. The function below:
1. Reads all whitespace-separated tokens from the file
2. Reshapes them into records of 61 columns each
3. Converts scientific notation strings to float values
4. Renames columns to human-readable element names

In [ ]:
def load_lpgrs_tab(filepath: str) -> pl.DataFrame:
    """
    Loads a Lunar Prospector GRS .tab file into a Polars DataFrame.
    
    Handles wrapped lines, multiple-space delimiters, and scientific notation.

    Args:
        filepath (str): Path to the .tab file.

    Returns:
        pl.DataFrame: DataFrame containing the GRS data with appropriate column names.
    """
    # Step 1: Read all whitespace-separated tokens
    # NASA records are often wrapped across multiple lines
    with open(filepath, 'r') as f:
        tokens = f.read().split()
    
    # Step 2: Reshape into 61 columns (standard format for GRS abundance files)
    # The 5-degree resolution file contains approximately 1,300 records
    record_width = 61
    rows = [tokens[i : i + record_width] for i in range(0, len(tokens), record_width)]
    
    # Step 3: Create DataFrame and cast scientific notation strings to Float64
    df = pl.DataFrame(rows, orient="row").select([
        pl.all().cast(pl.Float64)
    ])
    
    # Step 4: Define human-readable column names for the first 16 columns
    # These represent the key geochemical measurements from the GRS instrument
    names = [
        "bin_index",    # Unique identifier for each spatial bin
        "lat_start",    # Starting latitude of the bin (degrees)
        "lat_end",      # Ending latitude of the bin (degrees)
        "lon_start",    # Starting longitude of the bin (degrees)
        "lon_end",      # Ending longitude of the bin (degrees)
        "thorium",      # Thorium abundance (ppm) - key KREEP indicator
        "th_err",       # Thorium measurement uncertainty
        "potassium",    # Potassium abundance (wt%) - key KREEP indicator
        "k_err",        # Potassium measurement uncertainty
        "iron",         # Iron abundance (wt%) - high in mare basalts
        "fe_err",       # Iron measurement uncertainty
        "titanium",     # Titanium abundance (wt%) - high in high-Ti mare basalts
        "ti_err",       # Titanium measurement uncertainty
        "samarium",     # Samarium abundance (ppm) - rare earth element
        "sm_err",       # Samarium measurement uncertainty
        "calcium"       # Calcium abundance (wt%) - high in anorthosites
    ]
    
    # Select only the 16 columns we need and apply the new names
    return df.select(df.columns[:16]).rename(
        {old: new for old, new in zip(df.columns[:16], names)}
    )

In [ ]:
# Define paths to data files
# The dataset includes three file types:
#   .tab - The actual data (tabular format)
#   .lbl - Label file describing the data structure
#   .xml - Metadata in XML format
path_to_tab = "data/lpgrs_high1_elem_abundance_5deg.tab"
path_to_lbl = "data/lpgrs_high1_elem_abundance_5deg.lbl"
path_to_xml = "data/lpgrs_high1_elem_abundance_5deg.xml"

# Load the GRS data into a Polars DataFrame
df = load_lpgrs_tab(path_to_tab)

# Display the first few rows to verify the data loaded correctly
print(f"Loaded {df.height} records with {df.width} columns")
df.head()

---

## 3. Data Preparation & Normalization

Before applying machine learning algorithms, we need to prepare our data properly. This involves two critical steps:
1. **Data Cleaning**: Removing invalid or missing values
2. **Feature Scaling**: Normalizing the data to a common scale

### 3.1 Feature Selection & Data Cleaning

Machine learning models are sensitive to **"dirty" data**. Before we begin, we must ensure our dataset is clean.

We will filter our dataset to keep only the columns relevant to geochemistry:
- **Iron (Fe)** - Indicates mafic (basaltic) content
- **Titanium (Ti)** - Distinguishes high-Ti from low-Ti mare basalts
- **Thorium (Th)** - Key tracer for KREEP material
- **Potassium (K)** - Another KREEP indicator

> ⚠️ **Important:** We specifically exclude Latitude and Longitude from the training data because we want the model to group pixels based on **composition**, not their physical location.

In [ ]:
# Define the geochemical features we want to analyze
# These elements are diagnostic of different lunar rock types
feature_cols = ['iron', 'titanium', 'thorium', 'potassium']

# Drop rows where any of these values are null to prevent errors during ML training
# This ensures we only work with complete records
df_clean = df.drop_nulls(subset=feature_cols)

# Report how many rows were removed (if any)
print(f"Original dataset size: {df.height} rows")
print(f"Clean dataset size: {df_clean.height} rows")
print(f"Rows removed: {df.height - df_clean.height}")

### 3.2 Standardization (The "Apples to Oranges" Problem)

Geochemical data often comes in **mixed units**:
- **Iron (Fe)**: Measured in weight percent (`0-20 wt%`)
- **Thorium (Th)**: Measured in parts per million (`0-15 ppm`)

If we do not normalize this data, the K-Means algorithm will be **biased**. It will think Iron is "more important" simply because the numbers are larger.

We use `StandardScaler` to transform each feature to have:
- **Mean = 0**
- **Standard Deviation = 1**
Each row will be subtracted its mean $\mu$ and being divided by its Standard Deviation $\sigma$:

$ x = \frac{x - \mu}{\sigma}$

This forces all elements onto a level playing field, ensuring equal contribution to the clustering.

In [ ]:
# Import the StandardScaler from scikit-learn's preprocessing module
from sklearn.preprocessing import StandardScaler

# Convert the Polars DataFrame columns to a NumPy array
# Scikit-learn requires NumPy arrays as input
X_raw = df_clean.select(feature_cols).to_numpy()

# Initialize the StandardScaler
# This will compute mean and std for each feature
scaler = StandardScaler()

# Fit the scaler to our data and transform it in one step
# fit_transform() learns the parameters AND applies the transformation
X_scaled = scaler.fit_transform(X_raw)

# Verify the transformation worked correctly
print("Data standardization complete.")
print(f"\nExample comparison for first data point:")
print(f"  Raw Fe value:    {X_raw[0][0]:.4f} wt%")
print(f"  Scaled Fe value: {X_scaled[0][0]:.4f} (standardized)")
print(f"\nScaling parameters learned:")
print(f"  Feature means: {scaler.mean_}")
print(f"  Feature stds:  {scaler.scale_}")

---

## 4. The Elbow Method: Finding the Optimal Number of Clusters

How many distinct geochemical terranes exist on the Moon? Instead of guessing, we use the **"Elbow Method"**.

### How it works:
1. Run K-Means multiple times with different values of **k** (number of clusters)
2. Calculate the **Inertia** for each run (sum of squared distances from each point to its cluster center)
3. Plot inertia vs. k and look for the **"elbow"** — the point where adding more clusters provides diminishing returns

> 💡 **Tip:** The elbow represents the optimal trade-off between model complexity and clustering quality.

In [ ]:
# Import the K-Means clustering algorithm
from sklearn.cluster import KMeans

# List to store inertia values for each k
inertia = []

# Test k values from 1 to 9 clusters
k_range = range(1, 10)

print("Running Elbow Method test...")
print("-" * 40)

# Iterate through each value of k
for k in k_range:
    # Initialize KMeans with current k value
    # random_state=42 ensures reproducibility
    # n_init=10 means it runs 10 times with different random seeds and picks the best
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
    
    # Fit the model to our scaled data
    kmeans_temp.fit(X_scaled)
    
    # Store the inertia (within-cluster sum of squares)
    inertia.append(kmeans_temp.inertia_)
    
    # Print progress
    print(f"  k={k}: Inertia = {kmeans_temp.inertia_:.2f}")

print("-" * 40)
print("Elbow Method complete!")

# Create the Elbow plot
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia, marker='o', linestyle='-', color='b', linewidth=2, markersize=8)
plt.xlabel('Number of Clusters (k)', fontsize=12)
plt.ylabel('Inertia (Sum of Squared Distances)', fontsize=12)
plt.title('The Elbow Method: Determining Optimal Number of Clusters', fontsize=14)
plt.xticks(k_range)  # Show all k values on x-axis
plt.grid(True, alpha=0.3)

# Add annotation pointing to the likely elbow
plt.annotate('Elbow point', xy=(3, inertia[2]), xytext=(4.5, inertia[2] + 500),
             arrowprops=dict(arrowstyle='->', color='red'), fontsize=11, color='red')

plt.tight_layout()
plt.show()

---

## 5. Running the Classification Model

Based on the Elbow plot above, we see a clear bend around **k=3**. This aligns with the three major lunar terranes known from geological studies:

| Cluster | Terrane | Characteristics |
|---------|---------|-----------------|
| 1 | **Procellarum KREEP Terrane (PKT)** | High Th, High K, elevated Fe |
| 2 | **Feldspathic Highlands Terrane (FHT)** | Low Th, Low Fe, high Ca |
| 3 | **South Pole-Aitken Basin (SPA)** | Intermediate Fe, anomalous Th |

Now we run the final model with **k=3** and assign each pixel a "Geochemical Cluster ID."

In [ ]:
# Set the optimal number of clusters based on the Elbow plot analysis
k_optimal = 3 

# Initialize the final K-Means model
# random_state ensures reproducible results across runs
kmeans = KMeans(n_clusters=k_optimal, random_state=42, n_init=10)

# Fit the model and predict cluster labels in one step
# fit_predict() is equivalent to fit() followed by predict()
labels = kmeans.fit_predict(X_scaled)

# Attach the resulting Cluster IDs back to our clean DataFrame
# This allows us to visualize results geographically
df_results = df_clean.with_columns(
    pl.Series(name="Cluster_ID", values=labels)
)

# Summary statistics
print("=" * 50)
print("CLUSTERING COMPLETE")
print("=" * 50)
print(f"Number of clusters: {k_optimal}")
print(f"Total pixels classified: {len(labels)}")
print(f"\nPixels per cluster:")
for i in range(k_optimal):
    count = (labels == i).sum()
    percentage = count / len(labels) * 100
    print(f"  Cluster {i}: {count} pixels ({percentage:.1f}%)")

---

## 6. Visualization: The Machine Learning Geologic Map

Now we can visualize our results by plotting each pixel at its geographic location, colored by its assigned cluster.

**Key Questions to Consider:**
- Does the algorithm successfully identify the dark **Mare** regions (iron-rich basaltic plains)?
- Does it separate the bright **Highlands** (feldspathic, low-iron terrain)?
- Can you spot the **South Pole-Aitken Basin** (large impact basin on the far side)?

> 🔬 **Note:** The algorithm found these distinct regions based purely on chemistry, without ever knowing the latitude or longitude of the data points!

In [ ]:
# Create a figure with appropriate size for lunar map
plt.figure(figsize=(15, 8))

# Create the scatter plot using geographic coordinates
# Each point represents a 5-degree bin on the lunar surface
# Calculate the center and size of each bin for rectangle patches

# Create rectangle patches for each bin
patches = []
colors = []
for i in range(len(df_results)):
    row = df_results.row(i, named=True)
    width = row['lon_end'] - row['lon_start']
    height = row['lat_end'] - row['lat_start']
    rect = Rectangle((row['lon_start'], row['lat_start']), width, height)
    patches.append(rect)
    colors.append(row['Cluster_ID'])

# Create a PatchCollection with the rectangles
collection = PatchCollection(patches, cmap='viridis', alpha=0.8)
collection.set_array(colors)
collection.set_clim(0, 2)

# Add the collection to the axes
ax = plt.gca()
ax.add_collection(collection)
scatter = collection  # For colorbar compatibility

# Add colorbar to show cluster ID mapping
cbar = plt.colorbar(scatter, label='Geochemical Cluster ID')
cbar.set_ticks([0, 1, 2])
cbar.set_ticklabels(['Cluster 0', 'Cluster 1', 'Cluster 2'])

# Label the axes
plt.xlabel('Longitude (degrees)', fontsize=12)
plt.ylabel('Latitude (degrees)', fontsize=12)
plt.title(f'Unsupervised Geochemical Map of the Moon (k={k_optimal})', fontsize=14)

# Add grid for easier geographic reference
plt.grid(True, linestyle=':', alpha=0.6)

# Set axis limits to full lunar coverage
plt.xlim(-180, 180)
plt.ylim(-90, 90)

plt.tight_layout()
plt.show()

## 6.1 Scatter plot of two features plan

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
fig.set_figheight(5)
fig.set_figwidth(8)
ax.scatter(df_results["iron"], df_results["thorium"], s=16, c=df_results["Cluster_ID"], cmap='Paired', alpha=0.7) # c="white"
ax.set_xlabel('Iron (wt%)', fontsize=14)
ax.set_ylabel('Thorium (ppm)', fontsize=14)
ax.tick_params(axis='both', labelsize=12)
# Set the background color to a custom RGB value...
ax.set_facecolor('xkcd:salmon')
ax.set_facecolor((0.13, 0.13, 0.13))
fig.set_facecolor((0.13, 0.13, 0.13))
# Set the borders to a given color...
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')
ax.tick_params(color='white', labelcolor='white')
for spine in ax.spines.values():
    spine.set_edgecolor('white')
    spine.set_color('white')

---

## 7. Geological Interpretation: Decoding the Clusters

A map is useful, but we need to understand the **chemistry behind the colors**. By grouping the data by Cluster ID and calculating the mean of each element, we can create a "chemical fingerprint" for each terrane.

### Interpretation Guide:

| Chemistry Signature | Likely Terrane |
|---------------------|----------------|
| **High Th, High K, High Fe** | Procellarum KREEP Terrane (PKT) or Mare basalts |
| **Low Th, Low Fe, Low Ti** | Feldspathic Highlands Terrane (FHT) |
| **Intermediate Fe, Variable Th** | South Pole-Aitken Basin (SPA) |

> 📖 **KREEP** = Potassium (K), Rare Earth Elements (REE), Phosphorus (P) — a geochemical signature of the last dregs of the lunar magma ocean.

In [ ]:
# Calculate the mean chemistry for each cluster
# This creates a "fingerprint" for each geochemical terrane
cluster_stats = df_results.group_by("Cluster_ID").agg([
    pl.col("iron").mean().alias("Avg_Fe_wt%"),       # Average iron content
    pl.col("titanium").mean().alias("Avg_Ti_wt%"),   # Average titanium content
    pl.col("thorium").mean().alias("Avg_Th_ppm"),    # Average thorium content
    pl.col("potassium").mean().alias("Avg_K_wt%"),   # Average potassium content
    pl.len().alias("Pixel_Count")                     # Number of pixels in cluster
]).sort("Cluster_ID")

# Display the cluster statistics
print("=" * 60)
print("CLUSTER GEOCHEMICAL SIGNATURES")
print("=" * 60)
print(cluster_stats)
print("=" * 60)
print("\nCompare these values to known lunar terrane compositions to identify which cluster corresponds to which region!")

---

## 8. Conclusions & Next Steps

### What We Accomplished:
- Successfully loaded and processed NASA Lunar Prospector GRS data  
- Applied K-Means clustering to identify geochemically distinct regions  
- Created an automated geochemical map of the Moon  
- Identified three major terranes matching known lunar geology  

### Ideas for Further Exploration:
- Try different values of **k** (4, 5, 6) to see if sub-terranes emerge
- Experiment with other clustering algorithms (DBSCAN, Hierarchical Clustering)
- Add more elements to the feature set (Samarium, Calcium)
- Compare your results to published lunar terrane maps
- Apply the same technique to Mars GRS data!

---

*KMeans Workshop file complete! Continue with DBSCAN*